<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/Deploying_a_Hugging_Face_Model_with_Baseten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Baseten](https://assets.website-files.com/624535121db2930bcd043f5d/62453d9bddc3de287134cb76_baseten-logo.svg)

# Install Dependencies

In [ ]:
!pip install baseten
!pip install sentence-transformers

# Build and Test Sentence Transformer Model

In [ ]:
# We're using the all-MiniLM-L6-v2 model from Hugging Face

from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

# Write the Baseten-compatible Class with `load` and `predict` Methods 

In [ ]:
from sentence_transformers import SentenceTransformer

class SentenceTransformerModel(object):
    def load(self):
        self.model = SentenceTransformer(
            'sentence-transformers/all-MiniLM-L6-v2'
        )
    def predict(self, inputs):
        embeddings = self.model.encode(inputs)
        return embeddings

In [ ]:
# Sanity check the new Class-based model

stm = SentenceTransformerModel()
stm.load()
stm.predict(sentences)

# Generate Files to Deploy to Baseten

In [ ]:
!pip freeze | grep sentence-transformers > requirements.txt

In [ ]:
MODEL_CODE_AS_STR = """from sentence_transformers import SentenceTransformer

class SentenceTransformerModel(object):
    def load(self):
        self.model = SentenceTransformer(
            'sentence-transformers/all-MiniLM-L6-v2'
        )
    def predict(self, inputs):
        embeddings = self.model.encode(inputs)
        return embeddings
"""

with open('stm_model.py', 'w') as py_file:
    py_file.write(MODEL_CODE_AS_STR)

# Call Baseten API

In [ ]:
import baseten
baseten.login("*** INSERT API KEY ***")
baseten.deploy_custom(
    model_name='Sentence Transformer Embedding model',
    model_class='SentenceTransformerModel',
    model_files=['stm_model.py'],
    requirements_file='requirements.txt'
)